<a href="https://colab.research.google.com/github/mwzVI/Sentiment-Analysis/blob/main/sentiment_analysis_using_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB SENTIMENT ANALYSIS USING NLP 


You can download the dataset from the Kaggle link below: https://www.kaggle.com/ymanojkumar023/kumarmanoj-bag-of-words-meets-bags-of-popcorn 

Please download only: labeledTrainData.tsv file..


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords



In [ ]:
# We load our dataset here..
df = pd.read_csv('/content/drive/MyDrive/labeledTrainData.tsv',  delimiter="\t", quoting=3)



In [ ]:
# Let's look at the head of the dataset:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
len(df)

25000

In [ ]:
len(df["review"])

25000

In [ ]:
# Since stopwords aren't used in NLP we have to clear stopwords. For this purpose we must download stopword wordset from nltk library..
# We do this operation using nltk module..
nltk.download('stopwords')




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Data Cleaning Operations 

### First, we will delete HTML tags from review sentences using the BeautifulSoup module.

To explain how these processes are done, first let's choose a single review and see how it is done for you:

In [ ]:
sample = df.review[0]
sample

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [ ]:
# Clear HTML tags..
sample = BeautifulSoup(sample).get_text()
sample  # After the HTML tags are cleared.

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [ ]:
# we clean it from punctuation and numbers - using regex..
sample = re.sub("[^a-zA-Z]",' ',sample)
sample

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    m

In [ ]:
# we convert it to lowercase, since our machine learning algorithms think capitalized letters with different words
# and this may result mistakes..
sample = sample.lower()
sample

' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    m

In [ ]:
# stopwords (stopwords are the words like the, is, are not to be used by AI. These are grammar words..)
# First we split the words with split and convert them to a list. Our goal is to remove stopwords..
sample = sample.split()
  

In [ ]:
sample

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mj',
 's',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',

In [ ]:
len(sample)

437

In [ ]:
# sample without stopwords
swords = set(stopwords.words("english"))                      # conversion into set for fast searching
sample = [w for w in sample if w not in swords]               
sample


['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary',
 'watched',
 'wiz',
 'watched',
 'moonwalker',
 'maybe',
 'want',
 'get',
 'certain',
 'insight',
 'guy',
 'thought',
 'really',
 'cool',
 'eighties',
 'maybe',
 'make',
 'mind',
 'whether',
 'guilty',
 'innocent',
 'moonwalker',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'remember',
 'going',
 'see',
 'cinema',
 'originally',
 'released',
 'subtle',
 'messages',
 'mj',
 'feeling',
 'towards',
 'press',
 'also',
 'obvious',
 'message',
 'drugs',
 'bad',
 'kay',
 'visually',
 'impressive',
 'course',
 'michael',
 'jackson',
 'unless',
 'remotely',
 'like',
 'mj',
 'anyway',
 'going',
 'hate',
 'find',
 'boring',
 'may',
 'call',
 'mj',
 'egotist',
 'consenting',
 'making',
 'movie',
 'mj',
 'fans',
 'would',
 'say',
 'made',
 'fans',
 'true',
 'really',
 'nice',
 'actual',
 'feature',
 'film',
 'bit',
 'finally',
 'starts',
 'minutes',
 'excluding',
 'smooth',
 'crim

In [ ]:
len(sample)

219

- After describing the cleanup process, we now batch clean the reviews in our entire dataframe in a loop
- for this purpose we first create a function:

In [ ]:
def process(review):
    # without HTML tags
    review = BeautifulSoup(review).get_text()
    # without punctuation and numbers
    review = re.sub("[^a-zA-Z]",' ',review)
    # lowercase and splitting to eliminate stopwords
    review = review.lower()
    review = review.split()
    # without stopwords
    swords = set(stopwords.words("english"))                      
    review = [w for w in review if w not in swords]               
    # we join splitted paragraphs with join before return..
    return(" ".join(review))

In [ ]:
# We clean our training data with the help of the above function:
# We can see the status of the review process by printing a line after every 1000 reviews.

train_x_tum = []
for r in range(len(df["review"])):        
    if (r+1)%1000 == 0:        
        print("No of reviews processed =", r+1)
    train_x_tum.append(process(df["review"][r]))

No of reviews processed = 1000
No of reviews processed = 2000
No of reviews processed = 3000
No of reviews processed = 4000
No of reviews processed = 5000
No of reviews processed = 6000
No of reviews processed = 7000
No of reviews processed = 8000
No of reviews processed = 9000
No of reviews processed = 10000
No of reviews processed = 11000
No of reviews processed = 12000
No of reviews processed = 13000
No of reviews processed = 14000
No of reviews processed = 15000
No of reviews processed = 16000
No of reviews processed = 17000
No of reviews processed = 18000
No of reviews processed = 19000
No of reviews processed = 20000
No of reviews processed = 21000
No of reviews processed = 22000
No of reviews processed = 23000
No of reviews processed = 24000
No of reviews processed = 25000


### Train, test split...

In [ ]:
# Now we are going to split our data as train and test..
x = train_x_tum
y = np.array(df["sentiment"])

# train test split
train_x, test_x, y_train, y_test = train_test_split(x,y, test_size = 0.1)


### We are building our Bag of Words !

We have cleaned our data, but for the artificial intelligence to work, it is necessary to convert this text-based data into numbers and a matrix called bag of words. Here we use the CountVectorizer tool in sklearn for this purpose:

<IMG src="bag.jpg" width="900" height="900" >

In [ ]:
# Using the countvectorizer function in sklearn, we create a bag of words with a maximum of 5000 words...
vectorizer = CountVectorizer( max_features = 5000 )

# We convert our train data to feature vector matrix
train_x = vectorizer.fit_transform(train_x)



In [ ]:
train_x

<22500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1778285 stored elements in Compressed Sparse Row format>

In [ ]:
# We are converting it to array because it wants array for fit operation..
train_x = train_x.toarray()
train_y = y_train

In [ ]:
train_x.shape, train_y.shape

((22500, 5000), (22500,))

In [ ]:
train_y

array([1, 0, 1, ..., 1, 1, 1])

### We build and fit a Random Forest Model

In [ ]:
model = RandomForestClassifier(n_estimators = 100, random_state=42)
model.fit(train_x, train_y)


RandomForestClassifier(random_state=42)

### Now it's time for our test data..

In [ ]:
# We convert our test data to feature vector matrix
test_xx = vectorizer.transform(test_x)


In [ ]:
test_xx

<2500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 196770 stored elements in Compressed Sparse Row format>

In [ ]:
test_xx = test_xx.toarray()

In [ ]:
test_xx.shape

(2500, 5000)

#### Now let's predict..

In [ ]:
test_predict = model.predict(test_xx)
acc = roc_auc_score(y_test, test_predict)

In [ ]:
print("Accuracy: % ", acc * 100)

Accuracy: %  84.79690787170406
